# Creating and Evaluating Solutions <a class="anchor" id="top"></a>

In this notebook, you will train several models using Amazon Personalize.

1. [Introduction](#intro)
1. [Create solutions](#solutions)
1. [Evaluate solutions](#eval)
1. [Using evaluation metrics](#use)
1. [Storing useful variables](#vars)

## Introduction <a class="anchor" id="intro"></a>

To recap, for the most part, the algorithms in Amazon Personalize (called recipes) look to solve different tasks, explained here:

1. **HRNN & HRNN-Metadata** - Recommends items based on previous user interactions with items.
1. **HRNN-Coldstart** - Recommends new items for which interaction data is not yet available.
1. **Personalized-Ranking** - Takes a collection of items and then orders them in probable order of interest using an HRNN-like approach.
1. **SIMS (Similar Items)** - Given one item, recommends other items also interacted with by users.
1. **Popularity-Count** - Recommends the most popular items, if HRNN or HRNN-Metadata do not have an answer - this is returned by default.

No matter the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred

We also support event types and event values defined by:

1. **Event Type** - Categorical label of an event (browse, purchased, rated, etc).
1. **Event Value** - A value corresponding to the event type that occurred. Generally speaking, we look for normalized values between 0 and 1 over the event types. For example, if there are three phases to complete a transaction (clicked, added-to-cart, and purchased), then there would be an event_value for each phase as 0.33, 0.66, and 1.0 respectfully.

The event type and event value fields are additional data which can be used to filter the data sent for training the personalization model. In this particular exercise we will not have an event type or event value. 

To run this notebook, you need to have run the previous notebook, `01_Validating_and_Importing_User_Item_Interaction_Data`, where you created a dataset and imported interaction data into Amazon Personalize. At the end of that notebook, you saved some of the variable values, which you now need to load into this notebook.

In [1]:
%store -r

## Create solutions <a class="anchor" id="solutions"></a>
[Back to top](#top)

In this notebook, you will create solutions with the following recipes:

1. HRNN
1. SIMS
1. Personalized-Ranking

Since you have not imported any metadata, and there are no coldstart items in this dataset, this notebook will skip the HRNN-Metadata and HRNN-Coldstart recipes. The Popularity-Count recipe is the simplest solution available in Amazon Personalize and it should only be used as a fallback, so it will also not be covered in this notebook.

Similar to the previous notebook, start by importing the relevant packages, and set up a connection to Amazon Personalize using the SDK.

In [2]:
import time
from time import sleep
import json

import boto3

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In Amazon Personalize, a specific variation of an algorithm is called a recipe. Different recipes are suitable for different situations. A trained model is called a solution, and each solution can have many versions that relate to a given volume of data when the model was trained.

To start, we will list all the recipes that are supported. This will allow you to select one and use that to build your model.

In [4]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'status': 'ACTIVE',
   'creationDateTime'

The output is just a JSON representation of all of the algorithms mentioned in the introduction.

Next we will select specific recipes and build models with them.

### HRNN

HRNN (hierarchical recurrent neural network) is one of the more advanced recommendation models that you can use and it allows for real-time updates of recommendations based on user behavior. It also tends to outperform other approaches, like collaborative filtering. This recipe takes the longest to train, so let's start with this recipe first.

For our use case, using the LastFM data, we can use HRNN to recommend new artists to a user based on the user's previous artist tagging behavior. Remember, we used the tagging data to represent positive interactions between a user and an artist.

First, select the recipe by finding the ARN in the list of recipes above.

In [5]:
HRNN_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

#### Create the solution

First you create a solution using the recipe. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

In [6]:
hrnn_create_solution_response = personalize.create_solution(
    name = "personalize-poc-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = HRNN_recipe_arn
)

hrnn_solution_arn = hrnn_create_solution_response['solutionArn']
print(json.dumps(hrnn_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-southeast-2:002268754244:solution/personalize-poc-hrnn",
  "ResponseMetadata": {
    "RequestId": "bf01d5cb-4cf2-45b0-ab2a-812d249d52fe",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:09:56 GMT",
      "x-amzn-requestid": "bf01d5cb-4cf2-45b0-ab2a-812d249d52fe",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 40 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create many models and deploy them quickly. So we will set up the while loop for all of the solutions further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [7]:
hrnn_create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

In [8]:
hrnn_solution_version_arn = hrnn_create_solution_version_response['solutionVersionArn']
print(json.dumps(hrnn_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-2:002268754244:solution/personalize-poc-hrnn/f1383f2f",
  "ResponseMetadata": {
    "RequestId": "d25f261d-6fa5-4faa-9210-fa43474382d9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:09:56 GMT",
      "x-amzn-requestid": "d25f261d-6fa5-4faa-9210-fa43474382d9",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS


SIMS is one of the oldest algorithms used within Amazon for recommendation systems. A core use case for it is when you have one item and you want to recommend items that have been interacted with in similar ways over your entire user base. This means the result is not personalized per user. Sometimes this leads to recommending mostly popular items, so there is a hyperparameter that can be tweaked which will reduce the popular items in your results. 

For our use case, using the LastFM data, let's assume we pick a particular artist. We can then use SIMS to recommend other artists based on the tagging behavior of the entire user base. The results are not personalized per user, but instead, differ depending on the original artist we chose as our input.

Just like last time, we start by selecting the recipe.

In [9]:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

#### Create the solution

As with HRNN, start by creating the solution first. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

In [10]:
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-southeast-2:002268754244:solution/personalize-poc-sims",
  "ResponseMetadata": {
    "RequestId": "bb3fbc5f-2b61-40d1-9fb9-0b48170d4ee4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:09:56 GMT",
      "x-amzn-requestid": "bb3fbc5f-2b61-40d1-9fb9-0b48170d4ee4",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 35 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create many models and deploy them quickly. So we will set up the while loop for all of the solutions further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [11]:
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [12]:
sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-2:002268754244:solution/personalize-poc-sims/c9995c55",
  "ResponseMetadata": {
    "RequestId": "986d1477-955d-4d05-a0ad-514741ce7605",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:09:56 GMT",
      "x-amzn-requestid": "986d1477-955d-4d05-a0ad-514741ce7605",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalized Ranking

Personalized Ranking is an interesting application of HRNN. Instead of just recommending what is most probable for the user in question, this algorithm takes in a user and a list of items as well. The items are then rendered back in the order of most probable relevance for the user. The use case here is for filtering on genre for example, or when you have a broad collection that you would like better ordered for a particular user.

For our use case, using the LastFM data, we could imagine that a particular record label is paying us to recommend their artists to our users in a special promotion. Therefore, we know the list of artists we want to recommend, but we want to find out which of these artists each user will like most. We would use personalized ranking to re-order the list of artists for each user, based on their previous tagging history. 

Just like last time, we start by selecting the recipe.

In [13]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Create the solution

As with the previous solution, start by creating the solution first. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

In [14]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-southeast-2:002268754244:solution/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "83e117a9-b210-4496-9b9e-01acd389b073",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:09:57 GMT",
      "x-amzn-requestid": "83e117a9-b210-4496-9b9e-01acd389b073",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 35 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create many models and deploy them quickly. So we will set up the while loop for all of the solutions further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [15]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [16]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-2:002268754244:solution/personalize-poc-rerank/134bd652",
  "ResponseMetadata": {
    "RequestId": "0401c0c2-5b50-4f24-bcce-85ca5d6d8ec4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:09:57 GMT",
      "x-amzn-requestid": "0401c0c2-5b50-4f24-bcce-85ca5d6d8ec4",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### View solution creation status

As promised, how to view the status updates in the console:

* In another browser tab you should already have the AWS Console up from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `View dataset groups`.
* Click the name of your dataset group, most likely something with POC in the name.
* Click `Solutions and recipes`.
* You will now see a list of all of the solutions you created above,  including a column with the status of the solution versions. Once it is `Active`, your solution is ready to be reviewed. It is also capable of being deployed.

Or simply run the cell below to keep track of the solution version creation status.

In [17]:
in_progress_solution_versions = [
    hrnn_solution_version_arn,
    sims_solution_version_arn,
    rerank_solution_version_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solutio

### Hyperparameter tuning

Personalize offers the option of running hyperparameter tuning when creating a solution. Because of the additional computation required to perform hyperparameter tuning, this feature is turned off by default. Therefore, the solutions we created above, will simply use the default values of the hyperparameters for each recipe. For more information about hyperparameter tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

If you have settled on the correct recipe to use, and are ready to run hyperparameter tuning, the following code shows how you would do so, using SIMS as an example.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-hpo",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    performHPO=True
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

If you already know the values you want to use for a specific hyperparameter, you can also set this value when you create the solution. The code below shows how you could set the value for the `popularity_discount_factor` for the SIMS recipe.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-set-hp",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    solutionConfig = {
        'algorithmHyperParameters': {
            'popularity_discount_factor': '0.7'
        }
    }
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

## Evaluate solution versions <a class="anchor" id="eval"></a>
[Back to top](#top)

It should not take more than an hour to train all the solutions from this notebook. While training is in progress, we recommend taking the time to read up on the various algorithms (recipes) and their behavior in detail. This is also a good time to consider alternatives to how the data was fed into the system and what kind of results you expect to see.

When the solutions finish creating, the next step is to obtain the evaluation metrics. Personalize calculates these metrics based on a subset of the training data. The image below illustrates how Personalize splits the data. Given 10 users, with 10 interactions each (a circle represents an interaction), the interactions are ordered from oldest to newest based on the timestamp. Personalize uses all of the interaction data from 90% of the users (blue circles) to train the solution version, and the remaining 10% for evaluation. For each of the users in the remaining 10%, 90% of their interaction data (green circles) is used as input for the call to the trained model. The remaining 10% of their data (orange circle) is compared to the output produced by the model and used to calculate the evaluation metrics.

![personalize metrics](../static/imgs/personalize_metrics.png)

We recommend reading [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) to understand the metrics, but we have also copied parts of the documentation below for convenience.

You need to understand the following terms regarding evaluation in Personalize:

* *Relevant recommendation* refers to a recommendation that matches a value in the testing data for the particular user.
* *Rank* refers to the position of a recommended item in the list of recommendations. Position 1 (the top of the list) is presumed to be the most relevant to the user.
* *Query* refers to the internal equivalent of a GetRecommendations call.

The metrics produced by Personalize are:
* **coverage**: The proportion of unique recommended items from all queries out of the total number of unique items in the training data (includes both the Items and Interactions datasets).
* **mean_reciprocal_rank_at_25**: The [mean of the reciprocal ranks](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) of the first relevant recommendation out of the top 25 recommendations over all queries. This metric is appropriate if you're interested in the single highest ranked recommendation.
* **normalized_discounted_cumulative_gain_at_K**: Discounted gain assumes that recommendations lower on a list of recommendations are less relevant than higher recommendations. Therefore, each recommendation is discounted (given a lower weight) by a factor dependent on its position. To produce the [cumulative discounted gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) at K, each relevant discounted recommendation in the top K recommendations is summed together. The normalized discounted cumulative gain (NDCG) is the DCG divided by the ideal DCG such that NDCG is between 0 - 1. (The ideal DCG is where the top K recommendations are sorted by relevance.) Amazon Personalize uses a weighting factor of 1/log(1 + position), where the top of the list is position 1. This metric rewards relevant items that appear near the top of the list, because the top of a list usually draws more attention.
* **precision_at_K**: The number of relevant recommendations out of the top K recommendations divided by K. This metric rewards precise recommendation of the relevant items.

Let's take a look at the evaluation metrics for each of the solutions produced in this notebook. *Please note, your results might differ from the results described in the text of this notebook, due to the quality of the LastFM dataset.* 

### HRNN metrics

First, retrieve the evaluation metrics for the HRNN solution version.

In [18]:
hrnn_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_solution_version_arn
)

print(json.dumps(hrnn_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-2:002268754244:solution/personalize-poc-hrnn/f1383f2f",
  "metrics": {
    "coverage": 0.0438,
    "mean_reciprocal_rank_at_25": 0.0502,
    "normalized_discounted_cumulative_gain_at_10": 0.0825,
    "normalized_discounted_cumulative_gain_at_25": 0.0911,
    "normalized_discounted_cumulative_gain_at_5": 0.0597,
    "precision_at_10": 0.016,
    "precision_at_25": 0.0082,
    "precision_at_5": 0.0188
  },
  "ResponseMetadata": {
    "RequestId": "f369d7ae-5e5c-451b-a3be-9dc8a4bf0f4a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:46:01 GMT",
      "x-amzn-requestid": "f369d7ae-5e5c-451b-a3be-9dc8a4bf0f4a",
      "content-length": "408",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


The normalized discounted cumulative gain above tells us that at 5 items, we have less than a 6% chance (5.9% literally) in a recommendation being a part of a user's interaction history (in the hold out phase from training and validation). Around 4% of the recommended items are unique, and we have a precision of only 1% in the top 5 recommended items. 

This is clearly not a great model, but keep in mind that we had to use tagging data for our interactions because the listening data did not have timestamps associated with it. These results indicate that the tagging data may not be as relevant as we hoped. But, let's take a look at the results of the other solutions first.

### SIMS metrics

Now, retrieve the evaluation metrics for the SIMS solution version.

In [19]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-2:002268754244:solution/personalize-poc-sims/c9995c55",
  "metrics": {
    "coverage": 0.0856,
    "mean_reciprocal_rank_at_25": 0.0266,
    "normalized_discounted_cumulative_gain_at_10": 0.0302,
    "normalized_discounted_cumulative_gain_at_25": 0.0402,
    "normalized_discounted_cumulative_gain_at_5": 0.0213,
    "precision_at_10": 0.0104,
    "precision_at_25": 0.0065,
    "precision_at_5": 0.0104
  },
  "ResponseMetadata": {
    "RequestId": "b4c5e298-a4b6-4607-be33-0c1d259108cd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:46:01 GMT",
      "x-amzn-requestid": "b4c5e298-a4b6-4607-be33-0c1d259108cd",
      "content-length": "409",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In this example we are seeing a slightly lower precision at 5 items, a little over 1% at 1.04% this time. Effectively this is probably within the margin of error, but given that no effort was made to mask popularity, it may just be returning super popular results that a large volume of users have interacted with in some way. 

### Personalized ranking metrics

Now, retrieve the evaluation metrics for the personalized ranking solution version.

In [20]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-2:002268754244:solution/personalize-poc-rerank/134bd652",
  "metrics": {
    "coverage": 0.0021,
    "mean_reciprocal_rank_at_25": 0.0249,
    "normalized_discounted_cumulative_gain_at_10": 0.0399,
    "normalized_discounted_cumulative_gain_at_25": 0.0524,
    "normalized_discounted_cumulative_gain_at_5": 0.0268,
    "precision_at_10": 0.0074,
    "precision_at_25": 0.0057,
    "precision_at_5": 0.0069
  },
  "ResponseMetadata": {
    "RequestId": "d8b6cd6e-a2c4-452c-9cbb-1f209f884361",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 09 Apr 2020 04:46:01 GMT",
      "x-amzn-requestid": "d8b6cd6e-a2c4-452c-9cbb-1f209f884361",
      "content-length": "411",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Just a quick comment on this one, here we see a very low precision. As this is also based on HRNN, this result is unexpected, but it could just be caused by an unfortunate test set. 

## Using evaluation metrics <a class="anchor" id="use"></a>
[Back to top](#top)

It is important to use evaluation metrics carefully. There are a number of factors to keep in mind.

* If there is an existing recommendation system in place, this will have influenced the user's interaction history which you use to train your new solutions. This means the evaluation metrics are biased to favor the existing solution. If you work to push the evaluation metrics to match or exceed the existing solution, you may just be pushing the HRNN to behave like the existing solution and might not end up with something better.
* The HRNN Coldstart recipe is difficult to evaluate using the metrics produced by Amazon Personalize. The aim of the recipe is to recommend items which are new to your business. Therefore, these items will not appear in the existing user transaction data which is used to compute the evaluation metrics. As a result, HRNN Coldstart will never appear to perform better than the other recipes, when compared on the evaluation metrics alone. 

Keeping in mind these factors, the evaluation metrics produced by Personalize are generally useful for two cases:
1. Comparing the performance of solution versions trained on the same recipe, but with different values for the hyperparameters.
1. Comparing the performance of solution versions trained on different recipes (except HRNN Coldstart).

Properly evaluating a recommendation system is always best done through A/B testing while measuring actual business outcomes. Since recommendations generated by a system usually influence the user behavior which it is based on, it is better to run small experiments and apply A/B testing for longer periods of time. Over time, the bias from the existing model will fade.

## Storing useful variables <a class="anchor" id="vars"></a>
[Back to top](#top)

Before exiting this notebook, run the following cells to save the version ARNs for use in the next notebook.

In [21]:
%store hrnn_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn

Stored 'hrnn_solution_version_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'rerank_solution_version_arn' (str)
